<a href="https://colab.research.google.com/github/gamecicn/TF2_Study/blob/KaggleMinst/Copy_of_T2E1_Kaggle_Augment1_MINST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd 
import numpy as np
import tensorflow as tf

In [4]:
# Download Data
csv_train_file = '/content/drive/MyDrive/DS_data/MINSTHW/train.csv'
csv_test_file = '/content/drive/MyDrive/DS_data/MINSTHW/test.csv'

df_data = pd.read_csv(csv_train_file)
df_test = pd.read_csv(csv_test_file)

df_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Preprcessing

In [8]:
# Split train data to train and valid dataset
df_data_copy = df_data.copy()

df_train  = df_data_copy.sample(frac=0.99, random_state=0)
df_valid = df_data_copy.drop(df_train.index)

print("lenght of train is {}".format(len(df_train)))
print("lenght of valid is {}".format(len(df_valid)))

train_labels = df_train.pop('label')
valid_labels = df_valid.pop('label')

df_train.head()

#np.apply_along_axis(lambda x : np.reshape, 1, df_train.to_numpy(), 28, 28)
df_train = df_train.to_numpy().reshape(-1, 28 ,28, 1)
df_valid = df_valid.to_numpy().reshape(-1, 28 ,28, 1)

lenght of train is 41580
lenght of valid is 420


## Build the model

In [39]:
'''
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(28, (2, 2), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(48, (2, 2), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(48, (2, 2), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.summary()
'''

data_augmentation = tf.keras.Sequential([
  #layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  #tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
  tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode="nearest"),
  #tf.keras.layers.experimental.preprocessing.RandomHeight(factor=0.2),
  #tf.keras.layers.experimental.preprocessing.RandomWidth(factor=0.2)
])

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
    data_augmentation,
    tf.keras.layers.Conv2D(28, (5, 5), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.30),
    tf.keras.layers.Conv2D(48, (5, 5), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.30),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.summary()


Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_40 (Sequential)   (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 24, 24, 28)        728       
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 12, 12, 28)        0         
_________________________________________________________________
batch_normalization_45 (Batc (None, 12, 12, 28)        112       
_________________________________________________________________
dropout_28 (Dropout)         (None, 12, 12, 28)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 8, 8, 48)          33648     
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 4, 4, 48)        

### Training

In [40]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#model.summary()

history = model.fit(df_train, 
                    train_labels,
                    batch_size=64,
                    epochs=60)



Epoch 1/60
650/650 [==============================] - 5s 7ms/step - loss: 0.7316 - accuracy: 0.7643
Epoch 2/60
650/650 [==============================] - 4s 7ms/step - loss: 0.1311 - accuracy: 0.9588
Epoch 3/60
650/650 [==============================] - 4s 7ms/step - loss: 0.0927 - accuracy: 0.9704
Epoch 4/60
650/650 [==============================] - 4s 7ms/step - loss: 0.0810 - accuracy: 0.9744
Epoch 5/60
650/650 [==============================] - 4s 7ms/step - loss: 0.0709 - accuracy: 0.9782
Epoch 6/60
650/650 [==============================] - 5s 7ms/step - loss: 0.0685 - accuracy: 0.9793
Epoch 7/60
650/650 [==============================] - 4s 7ms/step - loss: 0.0692 - accuracy: 0.9792
Epoch 8/60
650/650 [==============================] - 4s 7ms/step - loss: 0.0536 - accuracy: 0.9836
Epoch 9/60
650/650 [==============================] - 4s 7ms/step - loss: 0.0528 - accuracy: 0.9839
Epoch 10/60
650/650 [==============================] - 4s 7ms/step - loss: 0.0506 - accuracy: 0.9834

### Validate

In [41]:
 val_loss, val_acc = model.evaluate(df_valid,  valid_labels, verbose=2)

14/14 - 0s - loss: 0.0154 - accuracy: 0.9929


## Make predictions

In [42]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(df_test.to_numpy().reshape(-1, 28, 28, 1))

res = pd.DataFrame(data = {"ImageId" : np.arange(1, 1+len(predictions)) , 
                           "Label"   : list(map(np.argmax, predictions))})

In [38]:
res.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [44]:
res.to_csv("/content/drive/MyDrive/output/Kaggle_minst_hw_cnn9.csv", index=False)